## Setup

In [ ]:
!git clone --recursive https://github.com/graphdeco-inria/gaussian-splatting.git 3dgs
%cd 3dgs

Cloning into '3dgs'...
remote: Enumerating objects: 859, done.
remote: Total 859 (delta 0), reused 0 (delta 0), pack-reused 859 (from 1)
Receiving objects: 100% (859/859), 78.64 MiB | 21.71 MiB/s, done.
Resolving deltas: 100% (489/489), done.
Submodule 'SIBR_viewers' (https://gitlab.inria.fr/sibr/sibr_core.git) registered for path 'SIBR_viewers'
Submodule 'submodules/diff-gaussian-rasterization' (https://github.com/graphdeco-inria/diff-gaussian-rasterization.git) registered for path 'submodules/diff-gaussian-rasterization'
Submodule 'submodules/fused-ssim' (https://github.com/rahul-goel/fused-ssim.git) registered for path 'submodules/fused-ssim'
Submodule 'submodules/simple-knn' (https://gitlab.inria.fr/bkerbl/simple-knn.git) registered for path 'submodules/simple-knn'
Cloning into '/content/3dgs/SIBR_viewers'...
remote: Enumerating objects: 3293, done.        
remote: Counting objects: 100% (322/322), done.        
remote: Compressing objects: 100% (174/174), done.        
remote: Tot

In [ ]:
!pip install -q plyfile
!pip install -q rasterio
# !pip install -q /content/3dgs/submodules/diff-gaussian-rasterization
!pip install -q /content/3dgs/submodules/simple-knn
!pip install -q /content/3dgs/submodules/fused-ssim/

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 101.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [ ]:
# Training acceleration
# !pip uninstall diff-gaussian-rasterization -y
%cd submodules/diff-gaussian-rasterization
# !rm -r build
!git checkout 3dgs_accel
!pip install .

/content/3dgs/submodules/diff-gaussian-rasterization
Previous HEAD position was 9c5c202 toggle antialiasing
Branch '3dgs_accel' set up to track remote branch '3dgs_accel' from 'origin'.
Switched to a new branch '3dgs_accel'
Processing /content/3dgs/submodules/diff-gaussian-rasterization
  Preparing metadata (setup.py) ... done
  Created wheel for diff_gaussian_rasterization: filename=diff_gaussian_rasterization-0.0.0-cp311-cp311-linux_x86_64.whl size=3517456 sha256=ecdcc2a45b7969584d5171d47ecd8156292aa34687895935b775a6d059a4a458
  Stored in directory: /root/.cache/pip/wheels/c4/53/3b/24ad1418ba382e5d37bb0c87d69f4a707baaa022d53f84f1b5
Successfully built diff_gaussian_rasterization


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
data_dir = "/content/drive/MyDrive/CS280 Homeworks/final/data" # Replace with location of data

## Data Processing

In [ ]:
import os
import glob
import math
import json
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import rasterio
from scipy.optimize import least_squares
import random

#### Extract RPC Data

In [ ]:
def extract_rpc_data(tif_path):
    """Extract RPC data from rgb TIF file"""
    with rasterio.open(tif_path) as src:
        # Get RPC data
        rpc_data = {}
        try:
            # Check if there are RPC tags
            if 'RPC' in src.tag_namespaces():
                rpc_tags = src.tags(ns='RPC')

                # Extract all RPC parameters
                rpc_data["LAT_OFF"] = float(rpc_tags.get("LAT_OFF", 0))
                rpc_data["LONG_OFF"] = float(rpc_tags.get("LONG_OFF", 0))
                rpc_data["HEIGHT_OFF"] = float(rpc_tags.get("HEIGHT_OFF", 0))
                rpc_data["LAT_SCALE"] = float(rpc_tags.get("LAT_SCALE", 1))
                rpc_data["LONG_SCALE"] = float(rpc_tags.get("LONG_SCALE", 1))
                rpc_data["HEIGHT_SCALE"] = float(rpc_tags.get("HEIGHT_SCALE", 1))
                rpc_data["LINE_OFF"] = float(rpc_tags.get("LINE_OFF", 0))
                rpc_data["SAMP_OFF"] = float(rpc_tags.get("SAMP_OFF", 0))
                rpc_data["LINE_SCALE"] = float(rpc_tags.get("LINE_SCALE", 1))
                rpc_data["SAMP_SCALE"] = float(rpc_tags.get("SAMP_SCALE", 1))

                # Parse coefficient arrays
                for coef in ["LINE_NUM_COEFF", "LINE_DEN_COEFF", "SAMP_NUM_COEFF", "SAMP_DEN_COEFF"]:
                    if coef in rpc_tags:
                        # Parse the coefficients from space-separated string
                        rpc_data[coef] = [float(val) for val in rpc_tags[coef].split()]
        except Exception as e:
            print(f"Error extracting RPC data from {os.path.basename(tif_path)}: {e}")
            return None

        return rpc_data

In [ ]:
def calculate_viewing_angles(rpc_data):
    """Calculate approximate satellite viewing angles from RPC data"""
    if not rpc_data:
        return None, None

    # Extract first-order coefficients for height
    try:
        line_z_coef = rpc_data["LINE_NUM_COEFF"][3]  # Coefficient for Z
        samp_z_coef = rpc_data["SAMP_NUM_COEFF"][3]  # Coefficient for Z

        # ToDo Check off_nadir_angle calculation
        # Estimate off-nadir angle
        gradient_magnitude = np.sqrt(line_z_coef**2 + samp_z_coef**2)
        off_nadir_angle = np.atan(gradient_magnitude) * (180/np.pi) * 15
        off_nadir_angle = min(45, max(0, off_nadir_angle))

        # ToDo Check azimuth calculation
        # Estimate azimuth angle
        azimuth = np.atan2(samp_z_coef, line_z_coef) * (180/np.pi)
        if azimuth < 0:
            azimuth += 360

        return off_nadir_angle, azimuth
    except (KeyError, IndexError) as e:
        print(f"Error calculating viewing angles: {e}")
        return None, None

In [ ]:
def estimate_camera_position(lat, lon, alt, off_nadir, azimuth, satellite_height=500000):
    """
    Estimate the camera position in 3D space with flipped x-axis

    Args:
        lat, lon, alt: The ground point location (degrees, degrees, meters)
        off_nadir: Off-nadir angle in degrees
        azimuth: Azimuth angle in degrees
        satellite_height: Approximate satellite height in meters

    Returns:
        tuple: (camera_x, camera_y, camera_z) in meters in local ENU coordinates
    """
    # Calculate slant range (distance from ground to satellite)
    slant_range = satellite_height / math.cos(math.radians(off_nadir))

    # Calculate direction vector components
    # X is East, Y is North, Z is Up
    x = slant_range * math.sin(math.radians(off_nadir)) * math.sin(math.radians(azimuth))
    y = slant_range * math.sin(math.radians(off_nadir)) * math.cos(math.radians(azimuth))
    z = slant_range * math.cos(math.radians(off_nadir))

    # Flip the X-axis
    x = -x

    return (x, y, z)

In [ ]:
def latlon_to_xyz(lat, lon, alt=0, ref_lat=None, ref_lon=None):
    """
    Convert latitude, longitude to local ENU coordinates with flipped x-axis

    Args:
        lat, lon: Point coordinates (degrees)
        alt: Altitude in meters
        ref_lat, ref_lon: Reference point (degrees), if None will use the input point

    Returns:
        tuple: (x, y, z) in meters in local ENU coordinates
    """
    # If no reference point specified, use the input point
    if ref_lat is None:
        print("No reference point specified, using input point as reference.")
        ref_lat = lat
    if ref_lon is None:
        print("No reference point specified, using input point as reference.")
        ref_lon = lon
    print(f"Reference point: {ref_lat}, {ref_lon}")

    # Earth radius in meters
    R = 6378137.0

    # Convert to radians
    lat_rad = math.radians(lat)
    lon_rad = math.radians(lon)
    ref_lat_rad = math.radians(ref_lat)
    ref_lon_rad = math.radians(ref_lon)

    # Calculate distance in east direction (x) - then flip it
    x = -(R * math.cos(ref_lat_rad) * (lon_rad - ref_lon_rad))

    # Calculate distance in north direction (y)
    y = R * (lat_rad - ref_lat_rad)

    # Z is just the altitude (relative to reference altitude)
    z = alt

    return (x, y, z)

In [ ]:
def create_rotation_matrix(off_nadir, azimuth):
    """
    Create a rotation matrix from off-nadir angle and azimuth
    with flipped x-axis orientation

    Args:
        off_nadir: Off-nadir angle in degrees
        azimuth: Azimuth angle in degrees

    Returns:
        numpy.ndarray: 3x3 rotation matrix and components
    """
    # Convert angles to radians
    off_nadir_rad = math.radians(off_nadir)
    azimuth_rad = math.radians(azimuth)

    # Create rotation matrices for each axis
    # First, set the camera to point downward (negative Z)
    R_nadir = np.array([
        [-1, 0, 0],   # Changed from -1 to 1 for x-axis flip
        [0, 1, 0],
        [0, 0, -1]  # Points downward
    ])

    # Rotation for off-nadir angle (rotation around x-axis)
    R_off_nadir = np.array([
        [1, 0, 0],
        [0, math.cos(off_nadir_rad), -math.sin(off_nadir_rad)],
        [0, math.sin(off_nadir_rad), math.cos(off_nadir_rad)]
    ])

    # Rotation for azimuth angle (rotation around z-axis)
    # Modified for flipped x-axis
    R_azimuth = np.array([
        [math.cos(azimuth_rad), math.sin(azimuth_rad), 0],  # Changed signs
        [math.sin(azimuth_rad), -math.cos(azimuth_rad), 0],
        [0, 0, 1]
    ])

    # Apply rotations in correct order
    R = R_azimuth @ R_off_nadir @ R_nadir

    return R, R_nadir, R_off_nadir, R_azimuth

In [ ]:
def process_multiple_images(folder_path, pattern, output_json="extrinsics.json", output_npz="selected_extrinsics.npz"):
    """Process multiple RGB images with x-axis flipped"""
    # Find all matching files
    rgb_files = glob.glob(os.path.join(folder_path, pattern))

    if not rgb_files:
        print(f"No files matching pattern '{pattern}' found in {folder_path}")
        return

    print(f"Found {len(rgb_files)} RGB images")

    # Create a table for results
    results = []

    # Process each image
    for rgb_file in rgb_files:
        file_name = os.path.basename(rgb_file)

        # Extract RPC data
        rpc_data = extract_rpc_data(rgb_file)

        if rpc_data:
            # Get location
            lat = rpc_data.get("LAT_OFF")
            lon = rpc_data.get("LONG_OFF")
            alt = rpc_data.get("HEIGHT_OFF")

            # Calculate viewing angles
            off_nadir, azimuth = calculate_viewing_angles(rpc_data)

            # Add to results
            results.append({
                "file_name": file_name,
                "latitude": lat,
                "longitude": lon,
                "altitude": alt,
                "off_nadir_angle": off_nadir,
                "azimuth": azimuth,
                "google_maps_format": f"{lat}, {lon}"
            })
        else:
            print(f"Could not extract RPC data from {file_name}")

    # Sort results by filename
    results.sort(key=lambda x: x['file_name'])

    # Calculate reference point (first image)
    ref_lat = results[0]['latitude']
    ref_lon = results[0]['longitude']
    ref_alt = results[0]['altitude']

    # Add camera positions
    for r in results:
        # Estimate camera position with flipped x-axis
        cam_x, cam_y, cam_z = estimate_camera_position(
            r['latitude'], r['longitude'], r['altitude'],
            r['off_nadir_angle'], r['azimuth']
        )

        # Convert ground point to local coordinates with flipped x-axis
        ground_x, ground_y, ground_z = latlon_to_xyz(
            r['latitude'], r['longitude'], r['altitude'],
            ref_lat, ref_lon
        )

        # Store camera position in local coordinates relative to the reference point
        r['camera_x'] = cam_x + ground_x
        r['camera_y'] = cam_y + ground_y
        r['camera_z'] = cam_z + ground_z

        # Store ground point in local coordinates
        r['ground_x'] = ground_x
        r['ground_y'] = ground_y
        r['ground_z'] = ground_z

    # Print results in a table format
    print("\nResults (sorted by filename):")
    print("-" * 100)
    print(f"{'File Name':<20} {'Latitude':<10} {'Longitude':<12} {'Off-Nadir':<10} {'Azimuth':<10} {'Camera X':<10} {'Camera Y':<10} {'Camera Z':<10}")
    print("-" * 100)

    for r in results:
        print(f"{r['file_name']:<20} {r['latitude']:<10.6f} {r['longitude']:<12.6f} {r['off_nadir_angle']:<10.2f} {r['azimuth']:<10.2f} {r['camera_x']:<10.2f} {r['camera_y']:<10.2f} {r['camera_z']:<10.2f}")

    # Create extrinsics and save as JSON
    extrinsics = read_extrinsics(results, output_json, output_npz)

    return results, ref_lat, ref_lon, ref_alt, extrinsics

In [ ]:
def read_extrinsics(results, json_file="extrinsics.json", npz_file="selected_extrinsics.npz"):
    """Return dictionary of extrinsics for images"""
    extrinsics_dict = {}

    for i, r in enumerate(results):
        # Extract view number from filename
        view_number = os.path.basename(r['file_name']).split('_')[2]
        index = int(view_number)

        # Get camera position
        t = np.array([r['camera_x'], r['camera_y'], r['camera_z']])

        # Create rotation matrix from off-nadir and azimuth with flipped x-axis
        R, R_nadir, R_off_nadir, R_azimuth = create_rotation_matrix(
            r['off_nadir_angle'], r['azimuth']
        )

        # In computer vision, we need the transformation from world to camera
        # This is the inverse of the camera pose in world coordinates
        R_world_to_cam = R.T
        t_world_to_cam = -R_world_to_cam @ t

        # Create the 3x4 extrinsic matrix [R|t]
        extrinsic = np.zeros((3, 4))
        extrinsic[:3, :3] = R_world_to_cam
        extrinsic[:3, 3] = t_world_to_cam

        # Use string keys for the dictionary
        extrinsics_dict[str(index)] = extrinsic.tolist()

        print(f"Created extrinsic for index {index}:")
        print(extrinsic)

    # Save in a more readable JSON format
    with open(json_file, 'w') as f:
        json.dump(extrinsics_dict, f, indent=2)
    print(f"Saved extrinsics to {json_file} (JSON format)")

    # Save the dictionary to NPZ file
    np.savez(npz_file, **extrinsics_dict)
    print(f"Saved extrinsics to {npz_file}")

    return extrinsics_dict

In [ ]:
from scipy.spatial.transform import Rotation as R
import math

# def get_intrinsics(results, extrinsics, json_file='intrinsics.json'):
#   img_text = ""
#   img_size = 2048
#   ground_width = 500 # in meters

#   intrinsics = {}

#   for i, r in enumerate(results):
#       view_number = os.path.basename(r['file_name']).split('_')[2]
#       index = int(view_number)
#       altitude = float(r['camera_z'])
#       fov_rad = (2 * math.atan(ground_width * math.cos(r['off_nadir_angle']) / (2 * altitude)))
#       if fov_rad < 0:
#         fov_rad = 2 * math.pi + fov_rad
#       f_x = f_y = img_size / (2 * math.tan(fov_rad / 2))
#       c_x = img_size / 2
#       c_y = img_size / 2

#       intrinsics[index] = {
#           "fov": fov_rad,
#           "f_x": f_x,
#           "f_y": f_y,
#           "c_x": c_x,
#           "c_y": c_y
#       }

#       img_text += f"{index} PINHOLE {img_size} {img_size} {f_x} {f_y} {c_x} {c_y}\n"
#       print(f"Intrinsic for index {index}:\n{intrinsics[index]}")
#   with open(os.path.join(os.getcwd(), "cameras.txt"), "w") as f_out:
#       f_out.write(img_text)

#   with open(json_file, 'w') as f:
#         json.dump(intrinsics, f, indent=2)
#   print(f"Saved intrinsics to {json_file} (JSON format)")

#   return intrinsics


def create_colmap_images_text(results, extrinsics):
  img_text = ""
  for i, r in enumerate(results):
      view_number = os.path.basename(r['file_name']).split('_')[2]
      index = int(view_number)
      flip_x = np.diag([-1, 1, 1])
      R_wc = np.array(extrinsics[str(index)])[:3,:3]
      C = np.array([r['camera_x'], r['camera_y'], r['camera_z']])
      R_cw = R_wc.T
      t_cw = -R_cw @ C

      R_cw = flip_x @ R_cw # To undo the x-axis flip

      qx, qy, qz, qw = R.from_matrix(R_cw).as_quat()
      tx, ty, tz = t_cw

      img_text += f"{index} {qw} {qx} {qy} {qz} {tx} {ty} {tz} {index} {r['file_name']}\n"
      img_text += f"\n"
  with open(os.path.join(os.getcwd(), "images.txt"), "w") as f_out:
      f_out.write(img_text)

In [ ]:
def get_transforms_json(scene_path):
  ext_int_path = os.getcwd()
  cams_path = os.path.join(ext_int_path, "cameras.txt")
  imgs_path = os.path.join(ext_int_path, "images.txt")
  images_dir = os.path.join(scene_path, "images")

  # Parse camera intrinsics
  with open(cams_path, "r") as f:
      for line in f:
          if line.startswith("#") or line.strip() == "":
              continue
          parts = line.strip().split()
          model = parts[1]
          width = int(parts[2])
          height = int(parts[3])
          fx = float(parts[4])
          fy = float(parts[5])

          fov_rad = 2 * math.atan(2048 / fy / 2)
          cx = float(parts[6])
          cy = float(parts[7])
          break  # assuming single camera model

  # Parse poses from images.txt
  frames = []
  with open(imgs_path, "r") as f:
      lines = f.readlines()
      for i in range(0, len(lines), 2):
          line = lines[i]
          if line.startswith("#") or line.strip() == "":
              continue
          parts = line.strip().split()
          image_id = int(parts[0])
          qw, qx, qy, qz = map(float, parts[1:5])
          tx, ty, tz = map(float, parts[5:8])
          image_name = parts[9]
          image_name = os.path.splitext(image_name)[0]

          # Quaternion to rotation matrix
          q = np.array([qw, qx, qy, qz], dtype=np.float64)
          R = np.array([
              [1 - 2*q[2]**2 - 2*q[3]**2,     2*q[1]*q[2] - 2*q[3]*q[0],     2*q[1]*q[3] + 2*q[2]*q[0]],
              [2*q[1]*q[2] + 2*q[3]*q[0], 1 - 2*q[1]**2 - 2*q[3]**2,         2*q[2]*q[3] - 2*q[1]*q[0]],
              [2*q[1]*q[3] - 2*q[2]*q[0],     2*q[2]*q[3] + 2*q[1]*q[0], 1 - 2*q[1]**2 - 2*q[2]**2]
          ])

          # COLMAP gives world-to-camera; invert to camera-to-world
          # R = R.T
          t = np.array([[tx], [ty], [tz]])
          c2w = np.eye(4)
          # c2w[:3, :3] = R
          # c2w[:3, 3:] = -R @ t
          c2w[:3, :3] = R
          c2w[:3, 3:] = t
          frame = {
              "file_path": f"input/{image_name}",
              "transform_matrix": c2w.tolist()
          }
          frames.append(frame)

  # Compose final transforms dict
  transforms = {
      "camera_angle_x": fov_rad,
      "fl_x": fx,
      "fl_y": fy,
      "cx": cx,
      "cy": cy,
      "w": width,
      "h": height,
      "camera_model": model,
      "frames": frames
  }

  # Write to transforms.json
  with open(os.path.join(scene_path, "transforms_train.json"), "w") as f:
      json.dump(transforms, f, indent=4)

  with open(os.path.join(scene_path, "transforms_train.json"), "r") as f:
      transforms = json.load(f)

  frames = transforms["frames"]
  random.shuffle(frames)

  # 90/10 train/test split
  split_idx = int(len(frames) * 0.9)
  train_frames = frames[:split_idx]
  test_frames = frames[split_idx:]

  transforms_train = {**transforms, "frames": train_frames}
  transforms_test = {**transforms, "frames": test_frames}

  # Save outputs
  with open(os.path.join(scene_path, "transforms_train.json"), "w") as f:
      json.dump(transforms_train, f, indent=4)

  with open(os.path.join(scene_path, "transforms_test.json"), "w") as f:
      json.dump(transforms_test, f, indent=4)

  print("Created transforms_train.json and transforms_test.json")


#### Extract and create data files

In [ ]:
%cd /content/3dgs

/content/3dgs


In [ ]:
from PIL import Image
import shutil

RGB_folder = os.path.join(data_dir, "JAX_068")
output_dir = os.path.join(os.getcwd(), "data", "input")
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(RGB_folder):
    if filename.lower().endswith('.tif') or filename.lower().endswith('.tiff'):
        input_path = os.path.join(RGB_folder, filename)
        output_path = os.path.join(output_dir, os.path.splitext(filename)[0] + '.jpg')
        with Image.open(input_path) as im:
            im.convert('RGB').save(output_path, 'JPEG')

In [ ]:
results, ref_lat, ref_lon, ref_alt, extrinsics = process_multiple_images(
    RGB_folder,
    "JAX_068_*.tif",
    "camera_extrinsics.json",
    "selected_extrinsics.npz"
)
# intrinsics = get_intrinsics(results, extrinsics, "camera_intrinsics.json")
create_colmap_images_text(results, extrinsics)
# get_transforms_json(output_dir) # If using NeRF data format

Found 19 RGB images
Reference point: 30.2992, -81.6397
Reference point: 30.2992, -81.6397
Reference point: 30.2992, -81.6397
Reference point: 30.2992, -81.6397
Reference point: 30.2992, -81.6397
Reference point: 30.2992, -81.6397
Reference point: 30.2992, -81.6397
Reference point: 30.2992, -81.6397
Reference point: 30.2992, -81.6397
Reference point: 30.2992, -81.6397
Reference point: 30.2992, -81.6397
Reference point: 30.2992, -81.6397
Reference point: 30.2992, -81.6397
Reference point: 30.2992, -81.6397
Reference point: 30.2992, -81.6397
Reference point: 30.2992, -81.6397
Reference point: 30.2992, -81.6397
Reference point: 30.2992, -81.6397
Reference point: 30.2992, -81.6397

Results (sorted by filename):
----------------------------------------------------------------------------------------------------
File Name            Latitude   Longitude    Off-Nadir  Azimuth    Camera X   Camera Y   Camera Z  
-----------------------------------------------------------------------------------

# Training

#### Use COLMAP to get initial camera parameters and point correspondences

**Prerequisite:** must have COLMAP installed and added to system Path environment variables

In [ ]:
!python convert.py -s data

#### Update images.bin with estimated parameters
After running COLMAP to get an initial estimate for SfM, update the binary extrinsics files with the estimates from the RPC model.

In [ ]:
!wget https://raw.githubusercontent.com/colmap/colmap/dev/scripts/python/read_write_model.py

In [ ]:
from read_write_model import Image, read_images_binary, write_images_binary, qvec2rotmat, rotmat2qvec
import numpy as np

output_path = os.path.join(colmapdatadir, "sparse/0/images.bin")

images = read_images_binary(output_path)

for img in images.values():
    img_id = img.id
    view_number = img.name.split('_')[2]
    index = int(view_number)

    with open("camera_extrinsics.json", "r") as f:
      # Load your custom extrinsic (R, t) for this image
      extrinsics_data = json.load(f)
      extrinsic = np.array(extrinsics_data[str(index)])
      R_wc = extrinsic[:3, :3]
      t_wc = extrinsic[:3, 3:]

      R_cw = R_wc.T  # Invert world-to-camera to camera-to-world
      t_cw = -R_cw @ t_wc

      qvec = rotmat2qvec(R_cw)
      tvec = t_cw

      images[img_id] = Image(id=img_id, qvec=qvec, tvec=tvec, camera_id=img.camera_id, name=img.name, xys=img.xys, point3D_ids=img.point3D_ids)

write_images_binary(images, output_path)

#### Depth Regularization

In [ ]:
!git clone https://github.com/DepthAnything/Depth-Anything-V2.git depth_anything
%cd depth_anything

In [ ]:
!python run.py --encoder vitl --pred-only --grayscale --img-path ../data --outdir ../data/depth_maps
%cd ..

In [ ]:
!python utils/make_depth_scale.py --base_dir data --depths_dir depth_maps

#### Running Training

Move the generated depth maps folder into your chosen COLMAP data folder.

In [ ]:
# Without depth regularization
!python train.py -s data --eval --exposure_lr_init 0.001 --exposure_lr_final 0.0001 --exposure_lr_delay_steps 5000 --exposure_lr_delay_mult 0.001 --train_test_exp

In [ ]:
# With depth regularization
!python train.py -s data --eval --exposure_lr_init 0.001 --exposure_lr_final 0.0001 --exposure_lr_delay_steps 5000 --exposure_lr_delay_mult 0.001 --train_test_exp -d depth_maps

Outputs will be stored under

```
output/<NAME OF FOLDER>
```

